# The prediction accuracy of subscribing to a game-related newsletter using gender, age, and played hours.

## Introduction

* This project will mainly focus on a dataset about how people play video games. It will use data from this dataset to calculate the prediction accuracy of different characteristics or behaviors. The goal is to identify which characteristics or behaviors are more predictive. In the future, these characteristics can be used to predict whether a player will subscribe to a game-related newsletter.

  
* According to the dataset provided by a research group in Computer Science at UBC, we will primarily analyze three characteristics and behaviors. The main research question of this project is: "Among gender, age, and hours played, which factor has the highest prediction accuracy for subscribing to a game-related newsletter?"

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
library(cowplot)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

### Data information
  - The data is collected by a research group in Computer Science at UBC, led by Frank Wood, which is about how people played the game.
  - This dataset includes seven variables: experience, subscribe, hashed email, played hours, name, gender, and age. It has 196 observations.
  - Quantitative variable: played hours, age;  Qualitative variable: experience, subscribe, hashed email, name, gender.
  - The meaning of the variables:
    1. experience: The experience level of each player, is classified as "Pro", Veteran", "Amateur", "Beginner", and "Regular" five level.
    2. subscribe: Whether the player subscribes to the game-related newsletter or not. True means they subscribe, otherwise it gonna be false.
    3. hashed email: The email address of these players.
    4. played_hours: The time (in hours) that played totally played.
    5. name: The name of these players.
    6. gender: The gender of these players.
    7. Age: The age of these players.
  - Since this project only analyzes how hours played, gender, and age predict subscription, we need to filter out the variables experience, hashedEmail, and name to tidy the data. At the same time, we also need to convert the data types of subscribe and name into factors. Finally, we need to drop all missing values from the dataset.

In [7]:
player <- read_csv("data/players.csv")
player

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,17
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


In [8]:
player <- player |>
select(-hashedEmail, -name,-experience)|>
drop_na(subscribe, played_hours,Age)|>
mutate(subscribe = as_factor(subscribe), gender = as_factor(gender)) # true is 1, false is 0
player

subscribe,played_hours,gender,Age
<fct>,<dbl>,<fct>,<dbl>
TRUE,30.3,Male,9
FALSE,0.0,Male,17
TRUE,0.7,Female,21
⋮,⋮,⋮,⋮
TRUE,0,Male,17
TRUE,0,Female,17
FALSE,0,Male,17


In [9]:
player_split <- initial_split(player, prop = 0.75, strata = subscribe)
player_training <- training(player_split)
player_testing <- testing(player_split)
player_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |> 
      set_engine("kknn") |>
      set_mode("classification")
gridvals <- tibble(neighbors = seq(from = 1, to = 10, by = 1))
player_vfold <- vfold_cv(data = player_training, v = 5, strata = subscribe)

In [19]:
player_recipe1 <- recipe( subscribe~ played_hours, data = player_training)|>
step_center(all_predictors())|>
step_scale(all_predictors())

player_workflow1 <- workflow()|>
add_recipe(player_recipe1)|>
add_model(player_spec)

player_results1 <- player_workflow1 |>
  tune_grid(resamples = player_vfold, grid = gridvals) |>
  collect_metrics()

player_min1 <- player_results1 |>
   filter(.metric == "accuracy") |>
   slice_max(mean)# No Answer - remove if you provide an answer
player_min1
player_results1

neighbors,.metric,.estimator,mean,n,std_err,.config
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
9,accuracy,binary,0.6145455,5,0.1002621,Preprocessor1_Model09
10,accuracy,binary,0.6145455,5,0.1002621,Preprocessor1_Model10


neighbors,.metric,.estimator,mean,n,std_err,.config
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,accuracy,binary,0.3978788,5,0.06092266,Preprocessor1_Model01
1,roc_auc,binary,0.3819444,5,0.08789593,Preprocessor1_Model01
2,accuracy,binary,0.3978788,5,0.06092266,Preprocessor1_Model02
⋮,⋮,⋮,⋮,⋮,⋮,⋮
9,roc_auc,binary,0.4888889,5,0.1122134,Preprocessor1_Model09
10,accuracy,binary,0.6145455,5,0.1002621,Preprocessor1_Model10
10,roc_auc,binary,0.5189815,5,0.1250051,Preprocessor1_Model10


In [20]:
k_min <- player_min1 |>
         slice(1)|>
         pull(neighbors)

player_best_spec1 <- nearest_neighbor(weight_func = "rectangular", neighbors = k_min) |>
         set_engine("kknn") |>
         set_mode("classification")

player_best_fit1 <- workflow() |>
         add_recipe(player_recipe1) |>
         add_model(player_best_spec1) |>
         fit(data = player_training)

player_summary1 <- player_best_fit1 |>
          predict(player_testing)|>
          bind_cols(player_testing) |>
          metrics(truth = subscribe, estimate = .pred_class)

In [21]:
player_summary1

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.8421053
kap,binary,0.6274510


In [12]:
player_recipe1 <- recipe( subscribe~ gender, data = player_training)


player_workflow1 <- workflow()|>
add_recipe(player_recipe1)|>
add_model(player_spec)

player_results1 <- player_workflow1 |>
  tune_grid(resamples = player_vfold, grid = gridvals) |>
  collect_metrics()

player_min1 <- player_results1 |>
   filter(.metric == "accuracy") |>
   slice_max(mean)# No Answer - remove if you provide an answer
player_min1
player_results1

neighbors,.metric,.estimator,mean,n,std_err,.config
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,accuracy,binary,0.329697,5,0.1174028,Preprocessor1_Model01
2,accuracy,binary,0.329697,5,0.1174028,Preprocessor1_Model02


neighbors,.metric,.estimator,mean,n,std_err,.config
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,accuracy,binary,0.3296970,5,0.11740283,Preprocessor1_Model01
1,roc_auc,binary,0.5166667,5,0.07837076,Preprocessor1_Model01
2,accuracy,binary,0.3296970,5,0.11740283,Preprocessor1_Model02
⋮,⋮,⋮,⋮,⋮,⋮,⋮
9,roc_auc,binary,0.4777778,5,0.08932186,Preprocessor1_Model09
10,accuracy,binary,0.2296970,5,0.05261439,Preprocessor1_Model10
10,roc_auc,binary,0.4777778,5,0.08932186,Preprocessor1_Model10
